<a href="https://www.kaggle.com/code/alyeko/london-restaurant-analysis-visualization-wyreapi?scriptVersionId=105591718" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Contents
1. [Importing needed packages](#section-one)
2. [Reading the restaurant data](#section-two)
3. [Visualizing restaurant data](#section-three)
4. [Cluster Analysis](#section-four)
5. [Geopandas and folium plots](#section-five)
    - [Buffer plot around point of interest](#subsection-five-a)
    - [Distance function](#subsection-five-b)
    - [Functions for folium plots](#subsection-five-c)
6. [Convex hull plot](#section-six)
7. [London tube data](#section-seven)
    - [Reading london tube data (stations and lines)](#subsection-seven-a)
8. [Final plot of restaurant point, convex hull, london tube stations and tube lines](#section-eight)

<a id="section-one"></a>
## Importing needed packages

In [ ]:
!pip install matplotlib-scalebar

In [ ]:
import os
import fiona
import folium
import requests
import matplotlib
#import osmnx as ox
import pandas as pd
import geopandas as gpd
import plotly.express as px
from pyproj import Transformer
from folium.plugins import Draw
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from folium.plugins import BeautifyIcon
from sklearn.cluster import MiniBatchKMeans
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.font_manager import FontProperties
from shapely.geometry import Point, LineString, Polygon

<a id="section-two"></a>
## Reading the restaurant data

In [ ]:
df = pd.read_csv('/kaggle/input/ldndata/rdata.csv') #path changes depending on where data is stored
df.head()

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.Geocode_Longitude, df.Geocode_Latitude))
gdf.head()

In [ ]:
gdf.crs = "EPSG:4326" #Adding crs information to geodataframe
gdf.plot(marker = '*', color = 'green') #Plotting the geodataframe
plt.rcParams['figure.figsize'] = [10, 10] 

<a id="section-three"></a>
## Visualizing restaurant data

In [ ]:
#Loading United Kingdom Data from gpd datasets
# load a sample geodataframe
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))  
uk = world[world['name'] == 'United Kingdom']

ax = uk.plot(color ='#e3bccf', edgecolor = 'blue')
plt.rcParams['figure.figsize'] = [10, 10] 

# for x, y, label in zip(uk.geometry.representative_point().x, uk.geometry.representative_point().y, africa_gdf.name):
#     ax.annotate(label, xy=(x, y))
    
gdf.plot(ax=ax, color = 'green')

#### From the plot above it can be seen that not all the points are in London, and will be removed...
#### To do this, we need another dataset that has the London spatial data information, ie London shapefile...

In [ ]:
london = gpd.read_file("/kaggle/input/ldndata/greater_london_const_region.shp")
london.crs #crs here is projected wherea~s points geodataframe ('gdf') is geographic crs as seen in the next cell

In [ ]:
gdf.crs

In [ ]:
london['geometry'].to_crs(epsg=4326)#, allow_override=True)
london_new = london.to_crs(epsg=4326)#, allow_override=True)
london_new.head()

Breakdown of spatial datasets we have so far...
1. gdf
2. london
3. london_new


In [ ]:
london_new.crs #geographic coordinate system

In [ ]:
# ax = london.plot()
# gdf.plot(ax=ax)

ax = gdf.plot(alpha=0.1, color='green')
london_new.plot(ax=ax, color = '#C6A619')

#### Plot is not very intuitive, london boundary is shown by a small yellow dot!

In [ ]:
london_new.plot(color='#C6A619')

From the london plot above, it is seen that the highest latitude value is about 51.7 and the lowest value about 51.2
the lowest longitude value is about -0.2 and the highest value about 0.4

In [ ]:
to_del = []
long_del_east = gdf[gdf['Geocode_Longitude'] > 0.4].index.to_list()
long_del_west = gdf[gdf['Geocode_Longitude'] <-0.5].index.to_list()
lat_del_upper = gdf[gdf['Geocode_Latitude'] >51.7].index.to_list()
lat_del_lower = gdf[gdf['Geocode_Latitude'] <51.2].index.to_list()

to_del.extend(long_del_east)
to_del.extend(long_del_west)
to_del.extend(lat_del_upper)
to_del.extend(lat_del_lower)
print(len(to_del))
to_del = set(to_del)
print(len(to_del))

In [ ]:
print(f'Before deletion of rows: {len(gdf)}')
gdf = gdf.drop(index=to_del)
print(f'After deletion of rows: {len(gdf)}')

In [ ]:
gdf.plot() #looks much better, and within the london area range

In [ ]:
ax = london_new.plot(edgecolor='blue',color='#F1C40F')
gdf.plot(ax=ax, marker='*')
plt.rcParams['figure.figsize']= [8, 8]  #Some points are shown outside the boundary of london but can be ignored for now

<a id="section-four"></a>
## Cluster Analysis 

#### Code for cluster analysis adapted from https://towardsdatascience.com/finding-and-visualizing-clusters-of-geospatial-data-698943c18fed

MiniBatchKMeans?

In [ ]:
ssd = []
for i in range(2, 26):
    km = MiniBatchKMeans(n_clusters=i)
    km.fit_predict(gdf[['Geocode_Longitude','Geocode_Latitude']])
    ssd.append(km.inertia_)

In [ ]:
k = range(2, 26)
plt.plot(k, ssd, marker='x', color='purple')
plt.xlabel('k values')
plt.ylabel('Inertia vaules')
plt.title('Elbow method to find the optimal k value')

#### From the plot above we can conclude that the optimal k value is 15 since that is when the inertia values start decreasing linearly

In [ ]:
cluster = MiniBatchKMeans(n_clusters=10, random_state=1, verbose=1, )
cluster.fit_predict(gdf[['Geocode_Longitude','Geocode_Latitude']])

In [ ]:
cluster.inertia_

In [ ]:
centers = cluster.cluster_centers_
centers

n_clusters = len(cluster.cluster_centers_)
n_clusters

In [ ]:
centers_gs = gpd.GeoSeries(map(Point, zip(centers[:,0], centers[:,1])), crs= 'epsg:4326')
centers_gs #coordinates of cluster points

In [ ]:
centers_gs.plot(marker='X', color='black', markersize=100) #These are what center points of clusters look like
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Center points of clusters only')

In [ ]:
##### clusters are plotted with this function as well(code adapted from https://stackoverflow.com/questions/28227340/kmeans-scatter-plot-plot-different-colors-per-cluster)
plt.scatter(gdf.Geocode_Longitude, gdf.Geocode_Latitude, c=[matplotlib.cm.Spectral(float(i) /10) for i in cluster.labels_]);

In [ ]:
#Plotting London shapefile
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_aspect('equal')
london_new.plot(ax=ax, alpha=0.4, edgecolor='darkgrey', color='lightgrey', zorder=1)

centers_gs.plot(ax=ax, color='black', marker='X', alpha=1, markersize=100, zorder=3)
gdf.plot(ax=ax, 
         alpha=0.5, 
         c=[matplotlib.cm.Spectral(float(i) /10) for i in cluster.labels_], 
         linewidth=0.8, zorder=2)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

#Adding cluster center label to legend
legend_drawn_flag = True
plt.legend(['Cluster centers'], frameon=legend_drawn_flag)

#Adding north arrow
x, y, arrow_length = 0.035, 0.96, 0.12
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=15, headwidth=35),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes) 


#Adding scalebar
scalebar = ScaleBar(2, "cm", length_fraction=0.25, location='lower center')
ax.add_artist(scalebar)   

plt.title('London Map showing restaurants grouped into 10 clusters')
#plt.savefig('imgs/screenshot2.PNG', dpi=300, bbox_inches="tight")

In [ ]:
gdf.geometry

<a id="section-five"></a>
## Geopandas and folium plots

In [ ]:
rest_name = input('Enter name of restaurant: ')
rest_name_gdf = gdf[gdf['BusinessName'].str.contains(rest_name)]
if len(rest_name_gdf) == 0:
    print(f'There were {len(rest_name_gdf)} {rest_name} restaurants found.')
else:
    print(f'There were {len(rest_name_gdf)} {rest_name} restaurants found which are shown below in the plot')
    ax = london_new.plot(edgecolor='#F1C40F',color='#F1C40F', alpha=0.5)
    rest_name_gdf.plot(ax=ax, marker='*', color='red')
    legend_drawn_flag = True
    plt.legend([f'{rest_name} restaurants'], frameon=legend_drawn_flag)

    #Adding north arrow
    x, y, arrow_length = 0.035, 0.96, 0.09
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                arrowprops=dict(facecolor='black', width=15, headwidth=35),
                ha='center', va='center', fontsize=20,
                xycoords=ax.transAxes) 


    #Adding scalebar
    scalebar = ScaleBar(2, "cm", length_fraction=0.25, location='lower center')
    ax.add_artist(scalebar) 
    plt.rcParams['figure.figsize']= [8, 8]

In [ ]:
map2 = folium.Map(location=[51.51363, -0.09599], tiles="cartodbpositron", zoom_start=10, )
add_markers_to_the_map(map2, rest_name_pjtd, color = 'orange', icon='gg-circle') 

<a id="subsection-five-a"></a>
### Buffer plot around point of interest

In [ ]:
rest_name_gdf.loc[[2360]].BusinessName

Drawing a buffer of 1000m around Grabthai Bangkok Cafe(our point of interest) to locate other thai restaurants within this area. The best way to visualize this buffer is in folium as it is more interactive...

In [ ]:
rest_name_gdf.loc[[2360]].geometry

In [ ]:
buf1000_gdf = gpd.GeoDataFrame(geometry = rest_name_gdf.loc[[2360]]['geometry'].to_crs('epsg:27700').buffer(1000), 
                           crs='epsg:27700')
ax2 = buf1000_gdf.plot(color='purple')
london.plot(ax=ax2, edgecolor='#F1C40F',color='#F1C40F', alpha=0.1) #using the london gdf as its coordinates are projected

In [ ]:
rest_name_gdf.crs #crs is wgs84

#### Because buffers and distance calculations are better using projected coordinate system(crs), rest_name_gdf crs must be converted

In [ ]:
buf1000_gdf.crs

In [ ]:
rest_name_gdf.crs

In [ ]:
intersection_MeshBlock = gpd.overlay(rest_name_gdf.to_crs('epsg:27700'), 
                                     buf1000_gdf, 
                                     how="intersection")
intersection_MeshBlock.plot()

In [ ]:
intersection_MeshBlock

rest_name_pjtd = rest_name_gdf.to_crs('epsg:27700')
rest_name_pjtd.head()

<a id="subsection-five-b"></a>
### Distance Function

In [ ]:
rest_name_gdf.to_crs('epsg:27700').head()

In [ ]:
gdf.head()

In [ ]:
gdf.to_crs('epsg:27700').head()  

In [ ]:
gdf.crs='epsg:4326'
gdf.crs

In [ ]:
# LineString(coordinates = [gdf.loc[0].geometry, gdf.loc[1].geometry])
# round(gdf.to_crs('epsg:27700').loc[0].geometry.distance(gdf.to_crs('epsg:27700').loc[1].geometry), 3)


In [ ]:
a = gdf.loc[[0]]
a
b = gdf.loc[[1]]
b

In [ ]:
LineString(coordinates = [Point(zip(gdf.loc[[0]].Geocode_Longitude, gdf.loc[[0]].Geocode_Latitude)), Point(zip(gdf.loc[[1]].Geocode_Longitude, gdf.loc[[1]].Geocode_Latitude))])

In [ ]:
gdf.crs='epsg:27700'
gdf.crs

In [ ]:
def create_linestring_gdf(gdframe, epsg=27700):
    """This function creates a line geodataframe from a point geodataframe"""
    gdframe = gdframe.reset_index(drop=True)
    gdframe = gdframe.drop(["geometry"], axis=1)
    indexes = gdframe.index.to_list()
    crs_info = int(input('What crs information would you like?\n1. WGS84'
                         '\n2. Projected CRS with epsg value passed: '))
    if crs_info == 1:
        linestrings = []
        for i in range(len(indexes)):
            a_lon, a_lat = gdframe.loc[indexes[i-1]].Geocode_Longitude, gdframe.loc[indexes[i-1]].Geocode_Latitude
            b_lon, b_lat = gdframe.loc[indexes[i]].Geocode_Longitude, gdframe.loc[indexes[i]].Geocode_Latitude
            linestrings.append(LineString(coordinates = [Point(a_lat, a_lon), Point(b_lat, b_lon)]))
        gdframe['geometry'] = linestrings                           
    
    elif crs_info == 2:
        linestrings = []
        transformer = Transformer.from_crs(4326, epsg)
        
        for i in range(len(indexes)):
            a_lon, a_lat = gdframe.loc[indexes[i-1]].Geocode_Longitude, gdframe.loc[indexes[i-1]].Geocode_Latitude
            b_lon, b_lat = gdframe.loc[indexes[i]].Geocode_Longitude, gdframe.loc[indexes[i]].Geocode_Latitude
            a_transformed = transformer.transform(a_lat, a_lon)
            b_transformed = transformer.transform(b_lat, b_lon)
            linestrings.append(LineString(coordinates = [Point(a_transformed[0], a_transformed[1]), Point(b_transformed[0], b_transformed[1])]))
        gdframe['geometry'] = linestrings    

        
    return gdframe


def calc_dist_bw_pts_in(gdframe, epsg):
    """Calculates the euclidian distance(saved in a new column called distances) between the previous point and the current point in the point dataframe passed
       First argument that must be passed is the point dataframe
       Second argument that must be passed is the epsg code, since for proper distance calculations, WGS84 coordinates must be converted to projected coordinates"""
    
    gdframe = gdframe.reset_index(drop=True)
    indexes = gdframe.index.to_list()
    distances = []
    option = int(input('Select an option'+
                       '\n1. LineStrings gdf with WGS84 crs'+
                       '\n2. LineString gdf with OSGB(epsg:27700) crs\n'))
    
    if option == 1: #geometry of line gdf must be converted from WGS84 to projected depeding on epsg that the user passes
        for i in range(len(indexes)):
            distances.append(round(gdframe.to_crs(f'epsg:{epsg}').loc[indexes[i-1]].geometry.distance(gdframe.to_crs(f'epsg:{epsg}').loc[indexes[i]].geometry), 2))                  
        gdframe['distances']=distances                    

    elif option == 2:
        for i in range(len(indexes)):
            distances.append(round(gdframe.loc[i]['geometry'].length, 2))                 
        gdframe['distances']=distances  
    
    return gdframe

In [ ]:
line = create_linestring_gdf(intersection_MeshBlock) #creating linegdf from the intersection between the 1000m buffer and the thai restaurnt gdf
line = calc_dist_bw_pts_in(line, epsg=27700)
line

In [ ]:
intersection_MeshBlock

In [ ]:
line

In [ ]:
line.loc[1]['geometry']

In [ ]:
line.loc[0]

line.plot?

In [ ]:
ax3 = buf1000_gdf.plot(alpha=0.1)
intersection_MeshBlock.plot(ax=ax3)
line.plot(ax=ax3, color='black', legend=True)
# legend_drawn_flag = True
# plt.legend(['Euclidean distance'], frameon=legend_drawn_flag)

In [ ]:
buf1000_gdf.geometry.centroid ##returns the centroid

<a id="subsection-five-c"></a>
### Functions for folium plots

In [ ]:
# rest_name_gdf
# restaurants = rest_name_gdf['geometry'].to_json()
# restaurants = folium.features.GeoJson(restaurants)
# my_map.add_child(restaurants)
# my_map 
my_map = folium.Map(location=[51.5074, 0.1272], tiles="OpenStreetMap", zoom_start=11,)
def add_markers_to_the_map(map_object, gdf, color, icon=None):  
    points = list(zip(zip(gdf.Geocode_Latitude, gdf.Geocode_Longitude),
                      gdf.BusinessName, 
                      gdf.RatingValue, 
                      gdf.PostCode))
    for point in points:         
            html =f"Name: {point[1]} <br>RatingValue: {str(point[2])} <br>PostCode: {point[3]}"
        
#           popup = folium.Popup(popup_text, autopan='False', parse_html=True, max_width=500)
            
            iframe= folium.IFrame(html)
            popup = folium.Popup(iframe,
                    min_width=250, 
                    max_width=250, 
                    parse_html=True)
            
            marker = folium.Marker(location=point[0], 
                                   popup=popup).add_to(map_object)
    return map_object

add_markers_to_the_map(my_map, rest_name_gdf, color = 'black', icon='gg-circle')

buf_1000m = folium.Circle(location=[rest_name_gdf.loc[2360].Geocode_Latitude, 
               rest_name_gdf.loc[2360].Geocode_Longitude], 
               radius=1000, #1000m radius
               color='green', 
               fill=True, 
               opacity=0.1,
               fill_opacity=0.6)
buf_1000m.add_to(my_map)

buf_4000m = folium.Circle(location=[rest_name_gdf.loc[2360].Geocode_Latitude, 
               rest_name_gdf.loc[2360].Geocode_Longitude], 
               radius=4000, #1000m radius
               color='yellow', 
               fill=True, 
               opacity=0.1,
               fill_opacity=0.4)
buf_4000m.add_to(my_map)

folium.PolyLine(locations=[(line.loc[0].Geocode_Latitude, line.loc[0].Geocode_Longitude), 
                           (line.loc[1].Geocode_Latitude, line.loc[1].Geocode_Longitude)], 
                tooltip=f"{round(line.loc[[0]].distances[0],2)}").add_to(my_map)

my_map #we can see the restaurants within our area of interest, together with thwir popups

#### Finding the distances between all points in 'buf_4000m'(the yellow buffered area in the folium plot above)
#### What will be done
- 1. Gdf containing the intersection of the the yellow buffered area and the points that fall within that area, will be created
- 2. The line gdf is created from the point gdf in step1
- 3. The euclidean distances between all points in the gdf will be calculated using the function created above (calc_dist_bw_pts_in)
- 4. It is finally plotted on the folium plot

In [ ]:
buf_4000m

In [ ]:
buf4000_gdf = gpd.GeoDataFrame(geometry = rest_name_gdf.loc[[2360]]['geometry'].to_crs('epsg:27700').buffer(4000), 
                           crs='epsg:27700')
buf4000_gdf.plot(color='#F9D742', alpha=0.75)

In [ ]:
intersectn_restgdf_400buf = gpd.overlay(rest_name_gdf.to_crs('epsg:27700'), 
                                     buf4000_gdf, 
                                     how="intersection")
intersectn_restgdf_400buf.plot(color='black')

In [ ]:
points_in_400m_buf = create_linestring_gdf(intersectn_restgdf_400buf)
points_in_400m_buf

In [ ]:
lines_in400m_buf = calc_dist_bw_pts_in(points_in_400m_buf, epsg=27700)
lines_in400m_buf

In [ ]:
lines_in400m_buf.plot(color='black')

In [ ]:
# rest_name_gdf
# restaurants = rest_name_gdf['geometry'].to_json()
# restaurants = folium.features.GeoJson(restaurants)
# my_map.add_child(restaurants)
# my_map 
#functions

def add_markers_to_the_map(map_object, gdf, colour, icon=None):  
    """Takes map_object(folium map), geodataframe, colour and icon and creates a improved folium map 
       with additional features(popups at the locations in the geodataframe with colour passed)"""
    points = list(zip(zip(gdf.Geocode_Latitude, gdf.Geocode_Longitude),
                      gdf.BusinessName, 
                      gdf.RatingValue, 
                      gdf.PostCode))
    for point in points:         
            html =f"Name: {point[1]} <br>RatingValue: {str(point[2])} <br>PostCode: {point[3]}"
#           popup = folium.Popup(popup_text, autopan='False', parse_html=True, max_width=500)
            
            iframe= folium.IFrame(html)
            popup = folium.Popup(iframe,
                    min_width=250, 
                    max_width=250, 
                    parse_html=True)
            
            marker = folium.Marker(location=point[0], 
                                   popup=popup, 
                                   #icon=folium.Icon(icon_size=(25, 25), color=colour, icon=icon, prefix='fa')).add_to(map_object)
                                  ).add_to(map_object)
    return map_object

def add_lines_to(map_object, line_gdf, color):
    """Takes folim  map and line geodataframe and adds line plot on folium map with their corresponding distances shown on tooltp"""
    indexes = line_gdf.index.to_list()
    for i in range(len(indexes)):
        loc = [(line_gdf.loc[indexes[i-1]].Geocode_Latitude, line_gdf.loc[indexes[i-1]].Geocode_Longitude), 
               (line_gdf.loc[indexes[i]].Geocode_Latitude, line_gdf.loc[indexes[i]].Geocode_Longitude)]
        folium.PolyLine(locations=loc, color=color,
                        tooltip=line_gdf.loc[[i]].distances[i]).add_to(map_object)
    return map_object
    
my_map = folium.Map(location=[51.5074, 0.1272], tiles="OpenStreetMap", zoom_start=11,)
add_markers_to_the_map(my_map, rest_name_gdf, colour = 'black', icon=None) #calling the function

buf_1000m = folium.Circle(location=[rest_name_gdf.loc[2360].Geocode_Latitude, 
               rest_name_gdf.loc[2360].Geocode_Longitude], 
               radius=1000, #1000m radius
               color='green', 
               fill=True, 
               opacity=0.1,
               fill_opacity=0.6)
buf_1000m.add_to(my_map)

buf_4000m = folium.Circle(location=[rest_name_gdf.loc[2360].Geocode_Latitude, 
               rest_name_gdf.loc[2360].Geocode_Longitude], 
               radius=4000, #1000m radius
               color='yellow', 
               fill=True, 
               opacity=0.1,
               fill_opacity=0.4)
buf_4000m.add_to(my_map)

add_lines_to(my_map, line, color='red')
add_lines_to(my_map, lines_in400m_buf, 'black')

draw = Draw()
draw.add_to(my_map)  #This gives the user the opportunity to draw on the map
folium.LayerControl(collapsed=False).add_to(my_map)
my_map
# my_map #we can see the restaurants within our area of interest, together with lines connecting them and their popups

In [ ]:
folium.Icon?

#### Alternatively,  the Draw function in folium has extra tools to calculate the distances between point, which is a simplification of the custom functions(create_linestring_gdf & calc_dist_bw_pts_in) created

<a id="section-six"></a>
## Convex hull plot

#### Adding a convex hull to the plot to show the border of the region of interest

#### convex hull code adapted from https://fcpython.com/visualisation/convex-hulls-football-python

In [ ]:
hull = ConvexHull(rest_name_gdf[['Geocode_Longitude','Geocode_Latitude']])
hull

In [ ]:
rest_name_gdf.plot(color='black')

In [ ]:
rest_name_gdf.iloc[5]['Geocode_Longitude'], rest_name_gdf.iloc[5]['Geocode_Latitude']

In [ ]:
hull.vertices

In [ ]:
plt.plot(rest_name_gdf.Geocode_Longitude, rest_name_gdf.Geocode_Latitude, 'o', color='black')

#Loop through each of the hull's simplices
linepts = []
for simplex in hull.simplices:
    #Draw a black line between each
    linepts.append([(rest_name_gdf.iloc[simplex[0]]['Geocode_Latitude'], rest_name_gdf.iloc[simplex[0]]['Geocode_Longitude']), (rest_name_gdf.iloc[simplex[1]]['Geocode_Latitude'], rest_name_gdf.iloc[simplex[1]]['Geocode_Longitude'])])

    
linepts

In [ ]:
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))  #adapted from https://stackoverflow.com/questions/24467972/calculate-area-of-polygon-given-x-y-coordinates

folium.PolyLine?

In [ ]:
for x in linepts:
       folium.PolyLine(locations=x).add_to(my_map)

In [ ]:
my_map

<a id="section-seven"></a>
## London Tube data

In [3]:
#url = 'https://api.tfl.gov.uk/Line/victoria/' #'https://api.tfl.gov.uk/Line/Mode/tube'
# response = requests.request("GET", url)
# print(response.text) 

In [2]:
# Using osmnx to obtain london network graph data
# ox.graph_from_place?
# place = ['London']
# G = ox.graph_from_place(place, retain_all=True, simplify = True, network_type='all')
# 
# fig, ax = ox.plot_graph(G, node_size=0, 
#                         dpi = 100,bgcolor = bgcolor,
#                         save = False, edge_color=roadCols,
#                         edge_linewidth=1, edge_alpha=1) # 

---------

<a id="subsection-seven-a"></a>
### Reading london tube data (stations and lines)

#### Tube data downloaded from downloaded from https://www.doogal.co.uk/london_stations

In [ ]:
tube_df = pd.read_csv('/kaggle/input/ldndata/London_tube_lines.csv')
tube_df.head()

In [ ]:
# gpd.read_file('London_Train_Lines.kml')

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
tube_linesgdf = gpd.read_file('/kaggle/input/ldndata/London_Train_Lines.kml', driver='KML')
tube_linesgdf.head()

In [ ]:
# gpd.read_file('London_Train_Lines.kml')

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
tube_stnsgdf = gpd.read_file('/kaggle/input/ldndata/london_tube_stations.kml', driver='KML')
tube_stnsgdf.head()

In [ ]:
[i for i in tube_linesgdf.loc[[0]].geometry[0].coords]

In [ ]:
tube_linesgdf.plot(color='red')
plt.title('London tube lines')

In [ ]:
#showing what the tube lines look like in the real world
tube_linesgdf.explore(color='red')  #if explore doesn't run update version of geopandas

In [8]:
#### Obtaining from and to locations from tube lines to be added to a folium feature group

[list(tube_linesgdf.apply(lambda row: row.loc[[i for i in tube_linesgdf['indexx']]]).geometry[index_no].coords) for index_no in tube_linesgdf.index]


In [ ]:
tube_stnsgdf.explore(color='red')  #showing what the points look like in the real world

In [ ]:
tube_stnsgdf.columns

In [6]:
### Connecting London tube lines and station points with restaurant data so that layers can be visualized

In [5]:
#### Creating and adding a feature group made of tube lines

In [ ]:
tube_linesgdf.head()

for idx, pt in enumerate([list(tube_linesgdf.geometry[index_no].coords) for index_no in tube_linesgdf.index]):
    print(tube_linesgdf.loc[idx]['Name'])

In [1]:
# fg1 = folium.FeatureGroup(name='Tube', overlay=False, control=True)
# for idx, pt in enumerate([list(tube_linesgdf.geometry[index_no].coords) for index_no in tube_linesgdf.index]):
#     location = [pt[0][:2][::-1], pt[1][:2][::-1]] #obtaining only long, lat
#     fg1.add_child(folium.PolyLine(location, color='red', tooltip=tube_linesgdf.loc[idx]['Name']))
#     fg1.add_to(my_map)

# #my_map.add_child(fg1)
# # themap.save('map.html')

# # base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
# # folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)


# #folium.LayerControl(collapsed=False).add_to(my_map)
# my_map.add_child(folium.LayerControl(collapsed=False))
# my_map

In [9]:
#### Creating and adding a feature group made of tube stations

In [ ]:
tube_stnsgdf.head()

In [ ]:
tstns_in_roi = gpd.overlay(tube_stnsgdf, 
                          buf4000_gdf.to_crs('epsg:4326'), 
                          how="intersection")
tstns_in_roi.plot(color='red')
plt.title('Tube stations in the area of interest')

In [ ]:
print(f'There are {len(tstns_in_roi)} tube stations in the area of interest i.e yellow buffer zone')

[c for c in tstns_in_poi.geometry[0].coords][0][:2][::-1]

folium.Marker?

In [ ]:
tlines_in_roi = gpd.overlay(tube_linesgdf, 
                          buf4000_gdf.to_crs('epsg:4326'), 
                          how="intersection")
tlines_in_roi.plot(color='red')
plt.title('Tube lines in the region of interest')

In [ ]:
tstns_in_roi.head()

<a id="section-eight"></a>
## Final plot of restaurant point, convex hull, london tube stations and tube lines

In [ ]:
for idx, tstn in enumerate([list(tstns_in_roi.geometry[index_no].coords) for index_no in tstns_in_roi.index]):
    location = list(tstn[0][:2][::-1]) #obtaining only long, lat
    
    # circle marker  'code adapted from https://stackoverflow.com/questions/60131314/folium-draw-star-marker'
    icon_circle = BeautifyIcon(
                  icon_shape='circle-dot', 
                  border_color='red', 
                  border_width=4.5,)
    folium.Marker(location, icon=icon_circle, tooltip=tstns_in_roi.loc[idx]['Name']).add_to(my_map)
       
my_map

In [ ]:
for idx, pt in enumerate([list(tlines_in_roi.geometry[index_no].coords) for index_no in tlines_in_roi.index]):
    location = [pt[0][:2][::-1], pt[1][:2][::-1]] #obtaining only long, lat
    folium.PolyLine(location, color='red', tooltip=tlines_in_roi.loc[idx]['Name']).add_to(my_map)

my_map

------------------------------------------